Test inspired by the workflow of these articles:

* https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/
* https://towardsdatascience.com/granger-causality-and-vector-auto-regressive-model-for-time-series-forecasting-3226a64889a6

Granger’s causality tests the null hypothesis that the coefficients of past values in the regression equation is zero.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df_raw = pd.read_csv('cleansed_data//all_consumption_metadata.csv', index_col=0, parse_dates=True,
                    dtype={'loc_id': 'str', 'consumption_kvah':'float32', 'temperature':'float32', 
                           'el_price':'float32', 'oil_price':'float32'})

In [3]:
df_raw.head(3)

,loc_id,consumption_kvah,temperature,el_price,oil_price
time,,,,,
2018-01-01 00:00:00,0,27.0,5.5,258.01001,543.415771
2018-01-01 01:00:00,0,27.5,5.0,258.98999,543.415771
2018-01-01 02:00:00,0,27.0,4.8,255.75000,543.415771


In [4]:
d = {}
loc_ids = df_raw.loc_id.unique()

for loc in loc_ids: d[loc] = df_raw[df_raw['loc_id']==loc]

## 1 make series stationary

In [5]:
subset = d['0'].drop(['loc_id', 'oil_price'], axis=1)
subset.head(3)

,consumption_kvah,temperature,el_price
time,,,
2018-01-01 00:00:00,27.0,5.5,258.01001
2018-01-01 01:00:00,27.5,5.0,258.98999
2018-01-01 02:00:00,27.0,4.8,255.75000


In [6]:
from statsmodels.tsa.stattools import adfuller

In [7]:
def pprint_adfuller(results):
    """
    The null hypothesis of the Augmented Dickey-Fuller is that 
    there _is_ a unit root, with the alternative that there is no
    unit root. If the pvalue is above a critical size, then we 
    cannot reject that there is a unit root. 
    
    I.e. Test statistics < Critical value implies stationary
    times series.
    """
    print('Dickey-Fuller test results:')
    print(f'Test statistics:\t{results[0]:.4f}')
    print(f"""Critical value 5%:\t{results[4]['5%']:.4f}""")

In [8]:
diff_subset = subset - subset.shift(-24)
diff_subset = diff_subset.resample('M').mean().dropna()

In [9]:
diff_t = adfuller(diff_subset['temperature'])
pprint_adfuller(diff_t)

Dickey-Fuller test results:
Test statistics:	-3.2978
Critical value 5%:	-3.0685


In [10]:
diff_e = adfuller(diff_subset['el_price'])
pprint_adfuller(diff_e)

Dickey-Fuller test results:
Test statistics:	-5.2207
Critical value 5%:	-2.9865


In [11]:
diff_c = adfuller(diff_subset['consumption_kvah'])
pprint_adfuller(diff_c)

Dickey-Fuller test results:
Test statistics:	-3.4865
Critical value 5%:	-2.9865


In [12]:
from statsmodels.tsa.stattools import grangercausalitytests

Wang, Professor Wei. (2013). Vertical Specialization and Trade Surplus in China: Chandos Asian Studies Series 48 (Chandos Asian Studies Series 1). Chandos Publishing.

In [13]:
def granger_matrix(dataframe, maxlag=6):
    """
    
    """
    
    df = pd.DataFrame(columns=dataframe.columns, index=dataframe.columns)
    for s in dataframe:
        for t in dataframe:
            r = grangercausalitytests(dataframe[[s, t]], maxlag=maxlag, verbose=False)
            df.loc[s, t] = min([f"""{v[0]['ssr_ftest'][1]:.4f}""" for _, v in r.items()])
            
    return df

In [14]:
granger_matrix(diff_subset, maxlag=7)

,consumption_kvah,temperature,el_price
consumption_kvah,1.0000,0.4705,0.1359
temperature,0.6728,1.0000,0.4038
el_price,0.2062,0.0196,1.0000


Cool!

In [15]:
granger_matrices = {}

for loc, df in d.items():
    subset = df.copy().drop(['loc_id', 'oil_price'], axis=1)
    
    diff_subset = subset - subset.shift(-24)
    diff_subset = diff_subset.resample('M').mean().dropna()
    
    maxlag = int((diff_subset.shape[0] - 1) / 3) - 1
    granger_matrices[loc] = granger_matrix(diff_subset, maxlag=maxlag)

In [16]:
for k, v in granger_matrices.items():
    print('P-values for customer', k)
    print(v)
    print()

P-values for customer 0
                 consumption_kvah temperature el_price
consumption_kvah           1.0000      0.4705   0.1359
temperature                0.6728      1.0000   0.4038
el_price                   0.2062      0.0196   1.0000

P-values for customer 1
                 consumption_kvah temperature el_price
consumption_kvah           1.0000      0.5828   0.0992
temperature                0.2452      1.0000   0.5881
el_price                   0.1390      0.0082   1.0000

P-values for customer 3
                 consumption_kvah temperature el_price
consumption_kvah           1.0000      0.0067   0.1714
temperature                0.1797      1.0000   0.5455
el_price                   0.1100      0.0151   1.0000

P-values for customer 4
                 consumption_kvah temperature el_price
consumption_kvah           1.0000      0.0063   0.3117
temperature                0.0000      1.0000   0.1860
el_price                   0.2653      0.0002   1.0000

P-values for custome

### Find out which hypotheses are rejected

In [17]:
temp = []
el = []

for k, v in granger_matrices.items():

    if float(v['temperature']['consumption_kvah']) < 0.05 : temp.append(k)
    if float(v['el_price']['consumption_kvah']) < 0.05 : el.append(k)


#### Temperature Granger causes consumption for...

In [18]:
temp

['3', '4', '10', '11', '18']

#### El prices Granger cause consumption for...

In [19]:
el

['12', '7', '14']

## Load oil and el prices

In [42]:
oil_el = pd.read_csv('cleansed_data//oil_el.csv', index_col=0, parse_dates=True)

In [43]:
w = pd.read_csv('cleansed_data//Weather.csv', index_col=0, parse_dates=True, dtype={'weather_station':'str', 'temperature':'float32'})
w.head(3)

,weather_station,temperature
2018-01-01 00:00:00,SN35210,-0.3
2018-01-01 01:00:00,SN35210,-0.5
2018-01-01 02:00:00,SN35210,-1.4


### ...and include mean weather across all locations

In [44]:
date_range = pd.date_range(start=min(w.index), end=max(w.index), freq='H')

In [45]:
mean_weather = pd.DataFrame(index=date_range, columns=['temperature'])
mean_weather['temperature'] = [np.nanmean(w.loc[ind]['temperature']) for ind in date_range]
mean_weather = mean_weather.resample(rule='D').mean()

In [50]:
w_oil_el = mean_weather.merge(oil_el, how='inner', left_index=True, right_index=True)
w_oil_el.head()

,temperature,price_oil,price_el
2018-01-01,3.643561,543.41580,255.16250
2018-01-02,1.049621,542.76430,311.53000
2018-01-03,1.673485,552.40753,276.98834
2018-01-04,0.898864,556.86420,311.56293
2018-01-05,-0.243561,549.75885,304.32626


In [51]:
diff_w_el_oil = (w_oil_el - w_oil_el.shift(-1)).resample('M').mean().dropna()

In [52]:
_ = adfuller(diff_w_el_oil['temperature'])
pprint_adfuller(_)

Dickey-Fuller test results:
Test statistics:	-5.7503
Critical value 5%:	-3.0685


In [53]:
_ = adfuller(diff_w_el_oil['price_el'])
pprint_adfuller(_)

Dickey-Fuller test results:
Test statistics:	-4.5743
Critical value 5%:	-2.9865


In [54]:
_ = adfuller(diff_w_el_oil['price_oil'])
pprint_adfuller(_)

Dickey-Fuller test results:
Test statistics:	-4.3177
Critical value 5%:	-2.9922


In [55]:
granger_matrix(diff_w_el_oil, maxlag=5)

,temperature,price_oil,price_el
temperature,1.0000,0.0762,0.4581
price_oil,0.2361,1.0000,0.0263
price_el,0.0022,0.1726,1.0000
